In [1]:
# Установка Python-библиотеки (Natural Language Toolkit) через менеджер пакетов pip
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python -m pip install --upgrade pip' command.


In [2]:
# Импорт модуля SentimentIntensityAnalyzer из nltk.sentiment.vader
# Использую словарь настроений VADER (Valence Aware Dictionary and sEntiment Reasoner),который предназначен для анализа тональности комментариев
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
data = pd.read_csv('fetch_data_from_sql.csv', sep=';', header=None, names=["ReviewID", "CustomerID", "ProductID", "ReviewDate","Rating", "ReviewText"])

In [4]:
data.head(10)

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."
5,6,81,1,2025-12-21,4,Customer support was very helpful.
6,7,16,1,2024-01-29,3,"Average experience, nothing special."
7,8,55,8,2024-08-15,5,The quality is top-notch.
8,9,3,13,2023-09-01,4,"I love this product, will buy again!"
9,10,78,6,2024-06-17,5,"Excellent product, highly recommend!"


In [5]:
# Проверяем размерность таблицы
data.shape

(1363, 6)

In [6]:
# Проверяем на пустые значения
data.isna().sum()

ReviewID      0
CustomerID    0
ProductID     0
ReviewDate    0
Rating        0
ReviewText    0
dtype: int64

In [7]:
# Убедимся, что лексикон VADER загружен
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /mnt/HC_Volume_18315164/home-jupyter/jupyter-s-
[nltk_data]     hazhdeu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
# Инициализация VADER-анализатора
sia = SentimentIntensityAnalyzer()

In [9]:
# Определяю функцию для расчета оценок настроений с помощью VADER
def calculate_sentiment(review):
    #Получим оценку настроений для комментария
    #Метод .polarity_scores(review) возвращает словарь с оценками тональности комментария
    sentiment = sia.polarity_scores(review)
    #Возвращает оценку, которая представляет собой оценку от -1 (наиболее отрицательная) до 1 (наиболее положительная).
    return sentiment['compound']

In [10]:
#Определяю функцию для категоризации настроений (sentiment category), используя как оценку настроений (sentiment score), так и рейтинг отзывов (rating)
def categorize_sentiment(score, rating):
    if score > 0.05:  # Позитивная тональность
        if rating >= 4:
            return 'Positive'  # Высокий рейтинг и позитивная тональность
        elif rating == 3:
            return 'Mixed Positive'  # Нейтральный рейтинг и позитивная тональность
        else:
            return 'Mixed Negative'  # Низский рейтинг и позитивная тональность
    elif score < -0.05:  # Негативная тональность
        if rating <= 2:
            return 'Negative'  # Низкий рейтинг и негативная тональность
        elif rating == 3:
            return 'Mixed Negative'  # Нейтральный рейтинг и негативная тональность
        else:
            return 'Mixed Positive'  # Высокий рейтинг и негативная тональность
    else:  # Нейтральная тональность
        if rating >= 4:
            return 'Positive'  # Высокий рейтинг и нейтральная тональность
        elif rating <= 2:
            return 'Negative'  # Низкий рейтинг и нейтральная тональность
        else:
            return 'Neutral'  # Нейтральный рейтинг и нейтральная тональность


In [11]:
# Определяю функцию для распределения оценок тональности в текстовые диапазоны.
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Очень позитивное мнение
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Умеренно позитивное мнение
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Умеренно негативное мнение
    else:
        return '-1.0 to -0.5'  # Очень негативное мнение

In [12]:
customer_reviews_df= data

In [13]:
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [14]:
# Добавление колонки с оценкой настроения
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [15]:
# Добавление колонки с категорией оценки настроения
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

In [16]:
# Добавление колонки с дипазоном распределения оценок настроения
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [20]:
customer_reviews_df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [19]:
from IPython.display import FileLink

# Сохранение файла
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)

# Создание ссылки для скачивания
FileLink('fact_customer_reviews_with_sentiment.csv')


/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-hazhdeu/DataAnalystPortfolioProject_PBI_SQL_Python_MarketingAnalytics/fact_customer_reviews_with_sentiment.csv